# Train and deploy sklearn model via mlflow 

In [ ]:
!pip install --upgrade gitpython

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# project settings
project_uri = prefix.joinpath("mlprojects", "sklearn-diabetes")

# azure ml settings
experiment_name = "using-mlflow-sklearn-tutorial"
compute_target = "cpu-cluster"

In [ ]:
import mlflow

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

In [ ]:
backend_config = {"COMPUTE": compute_target}

In [ ]:
run = mlflow.projects.run(
    uri=str(project_uri),
    parameters={"alpha": 0.3},
    backend="azureml",
    backend_config=backend_config,
)
run

## Deploy to ACI or AKS

In [ ]:
from azureml.core.webservice import AksWebservice

aks_config = AksWebservice.deploy_configuration(
    compute_target_name="aks-cpu-deploy",
    cpu_cores=2,
    memory_gb=5,
    tags={"data": "diabetes", "method": "sklearn"},
    description="Predict using webservice",
)

In [ ]:
import mlflow.azureml
from random import randint

webservice, azure_model = mlflow.azureml.deploy(
    model_uri=f"runs:/{run.run_id}/model",
    workspace=ws,
    # deployment_config=aks_config, #  uncomment to deploy to AKS
    service_name="sklearn-diabetes-" + str(randint(10000, 99999)),
    model_name="sklearn-diabetes-example",
)

## Test webservice

In [ ]:
import pandas as pd

data = pd.read_csv(prefix.joinpath("data", "raw", "diabetes", "diabetes.csv"))

sample = data.drop(["progression"], axis=1).iloc[[0]]

query_input = sample.to_json(orient="split")
query_input = eval(query_input)
query_input.pop("index", None)

In [ ]:
headers = {
    "Content-Type": "application/json",
}

In [ ]:
import requests
import json

response = requests.post(
    url=webservice.scoring_uri, data=json.dumps(query_input), headers=headers
)
print(response.text)

## Delete webservice

In [ ]:
webservice.delete()